# Automated Feature Engineering in Python
This notebook is based on the tutorial: https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219

In [3]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [25]:
# Read in the data
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [26]:
# Group loans by client id and calculate mean, max, min of loans
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']

# Merge with the clients dataframe
stats = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left')

stats.head(10)

,client_id,joined,income,credit_score,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,8951.600000,14049,559
1,49545,2007-11-14,104564,770,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,7894.850000,14399,811
3,46180,2001-11-06,43851,562,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,7424.050000,14575,904
6,32726,2006-05-01,235705,730,6633.263158,14802,851
7,35089,2010-03-01,131176,771,6939.200000,13194,773
8,35214,2003-08-08,95849,696,7173.555556,14767,667
9,48177,2008-06-09,190632,769,7424.368421,14740,659


In [27]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [28]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [29]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.600000,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.850000,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,10,11.943883,7424.050000,14575,904
6,32726,2006-05-01,235705,730,5,12.370336,6633.263158,14802,851
7,35089,2010-03-01,131176,771,3,11.784295,6939.200000,13194,773
8,35214,2003-08-08,95849,696,8,11.470529,7173.555556,14767,667
9,48177,2008-06-09,190632,769,6,12.158100,7424.368421,14740,659


In [30]:
es = ft.EntitySet(id = 'clients')

In [31]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [32]:

# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [33]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [35]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [37]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [39]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [41]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation']

,name,type,description
0,std,aggregation,Computes the average value of a numeric feature.
1,count,aggregation,Computes the average value of a numeric feature.
2,all,aggregation,Computes the average value of a numeric feature.
3,last,aggregation,Computes the average value of a numeric feature.
4,n_most_common,aggregation,Computes the average value of a numeric feature.
5,min,aggregation,Computes the average value of a numeric feature.
6,trend,aggregation,Computes the average value of a numeric feature.
7,max,aggregation,Computes the average value of a numeric feature.
8,sum,aggregation,Computes the average value of a numeric feature.
9,median,aggregation,Computes the average value of a numeric feature.


In [42]:
primitives[primitives['type'] == 'transform']

,name,type,description
19,and,transform,
20,greater_than_equal_to_scalar,transform,
21,modulo_numeric,transform,
22,greater_than_equal_to,transform,
23,is_null,transform,
24,divide_by_feature,transform,
25,second,transform,
26,less_than_equal_to_scalar,transform,
27,add_numeric_scalar,transform,
28,year,transform,


In [46]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['year', 'month', 'subtract_numeric', 'divide_numeric'])

In [47]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
25707,10
26326,5
26695,8
26945,11
29841,8


In [48]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
25707,1178.552795
26326,1166.736842
26695,1207.433824
26945,1109.473214
29841,1439.433333


In [49]:
features.head()

,credit_score,income,log_income,join_month,credit_score / income,MEAN(loans.loan_amount),LAST(loans.rate),income / join_month,LAST(loans.loan_amount),MAX(loans.loan_amount),...,income - LAST(loans.rate),join_month - log_income / MEAN(loans.rate),LAST(loans.rate) - log_income,MEAN(loans.rate) / credit_score - income,MAX(loans.rate) / income - log_income,credit_score - join_month / credit_score - income,MEAN(loans.loan_amount) / MAX(loans.rate),log_income - MEAN(payments.payment_amount),credit_score - log_income / income,credit_score - join_month / LAST(loans.loan_amount)
client_id,,,,,,,,,,,,,,,,,,,,,
25707,621,211422,12.261611,10,0,7963.950000,7.40,21142,2203,13913,...,211414.60,-0.650449,-4.861611,-0.000016,0.000045,-1,843.638771,-1166.291184,0.002879,0
26326,633,227920,12.336750,5,0,7270.062500,1.45,45584,5275,13464,...,227918.55,-2.914300,-10.886750,-0.000011,0.000030,-1,1080.247028,-1154.400092,0.002723,0
26695,680,174532,12.069863,8,0,7824.722222,0.90,21816,13918,14865,...,174531.10,-1.650316,-11.169863,-0.000014,0.000037,-1,1201.954258,-1195.363960,0.003827,0
26945,806,214516,12.276140,11,0,7125.933333,2.86,19501,9249,14593,...,214513.14,-0.446932,-9.416140,-0.000013,0.000026,-1,1261.227139,-1097.197075,0.003700,0
29841,523,38354,10.554614,8,0,9813.000000,5.09,4794,7223,14837,...,38348.91,-0.741543,-5.464614,-0.000091,0.000176,-1,1451.627219,-1428.878719,0.013361,0


In [50]:
# Show a feature with a depth of 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
25707,293.500000
26326,977.375000
26695,1769.166667
26945,1598.666667
29841,1125.500000
32726,799.500000
32885,1729.000000
32961,282.600000
35089,110.400000


In [51]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [52]:
features.iloc[:, 4:].head()

,MEAN(loans.loan_amount),NUM_UNIQUE(loans.repaid),NUM_UNIQUE(loans.loan_type),COUNT(loans),STD(loans.rate),MODE(payments.missed),MEAN(payments.payment_amount),MAX(loans.loan_amount),MAX(payments.payment_amount),COUNT(payments),...,MEAN(loans.MIN(payments.payment_amount)),SKEW(loans.NUM_UNIQUE(payments.missed)),STD(loans.NUM_UNIQUE(payments.missed)),MODE(loans.YEAR(loan_start)),MIN(loans.COUNT(payments)),SKEW(loans.MIN(payments.payment_amount)),MAX(loans.SUM(payments.payment_amount)),SUM(loans.NUM_UNIQUE(payments.missed)),MEAN(loans.MEAN(payments.payment_amount)),MEAN(loans.MAX(payments.payment_amount))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,7963.950000,2,4,20,2.484186,1,1178.552795,13913,2704,161,...,815.050000,0.0,0.0,2010,5,0.190578,19246,40,1175.760996,1578.750000
26326,7270.062500,2,4,16,2.057142,1,1166.736842,13464,2658,133,...,829.500000,0.0,0.0,2003,5,0.350923,19295,32,1161.691608,1547.500000
26695,7824.722222,2,4,18,1.561659,0,1207.433824,14865,2932,136,...,944.888889,0.0,0.0,2003,5,0.695612,22686,36,1221.200615,1518.833333
26945,7125.933333,2,4,15,1.619717,1,1109.473214,14593,2768,112,...,820.866667,0.0,0.0,2002,5,0.203092,20181,30,1115.150112,1411.600000
29841,9813.000000,2,4,18,2.122904,1,1439.433333,14837,2898,150,...,1002.166667,0.0,0.0,2005,6,0.098515,21454,36,1455.233737,1866.000000
